Users & Devices
===================================================

Cleaning up list of devices present at ITU for the entire period
We will look for:
- devices always connected to 1-3 devices in the entire period
- devices present for 'suspiciously long time


In [1]:
from lib.cached_data_loader import CachedDataLoader
from lib.panda_plotter import PandaPlotter
cdl = CachedDataLoader()
pp = PandaPlotter()


In [3]:
# COUNT DISTINCT DEVICES
sql = "SELECT COUNT(DISTINCT(wc.client_mac)) AS DEVICE_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df)

   DEVICE_COUNT
0          9113


In [4]:
# COUNT DISTINCT USERS
sql = "SELECT COUNT(DISTINCT(wc.user_name)) AS USER_COUNT FROM wifi_clients AS wc;"
df = cdl.get_dataframe(sql)
print(df)

   USER_COUNT
0        7267


In [5]:
# DEVICES CONNECTED TO MAX 3 Access Points
sql = "SELECT wc.client_mac, " \
      "COUNT(DISTINCT(wc.device_id)) AS DEVCNT, " \
      "COUNT(DISTINCT(wc.mqtt_batch)) AS BATCHES " \
      "FROM wifi_clients AS wc " \
      "GROUP BY wc.client_mac " \
      "HAVING COUNT(DISTINCT(wc.device_id)) <= 3 " \
      "ORDER BY COUNT(DISTINCT(wc.mqtt_batch)) DESC"
df = cdl.get_dataframe(sql)
print(df)


                            client_mac  DEVCNT  BATCHES
0     NnA3K1RKTGxvK3dhVm5abFBBa2J4dz09       2    13022
1     ejdiWGpPdExMaGdWZHF5eDIvVHd1dz09       3    13020
2     cWdsWnpsazNsV1JPZHEzanFOemorZz09       2    13000
3     RWdsNnByclZJOGdMWW40YUN2U3JRUT09       3    12552
4     TVJ0eGlYWTVENXlTWHVxOFlyYTJOUT09       2    11110
...                                ...     ...      ...
3777  RjNndm92UkMweFViS0oyV3E1c01RUT09       1        1
3778  NzdkTHJRR05sNWFOdFlXWUFtSnhhdz09       1        1
3779  RkNlVndxcm4wUmZndmVxSmMwTkdWZz09       1        1
3780  ODlnUlVMamJxajZHcEZsVjJOZUJHQT09       1        1
3781  ODZady9JV0NJK3NBMXVraTRVczladz09       1        1

[3782 rows x 3 columns]
